In [57]:
import json 
import pandas as pd 
import numpy as np 

In [184]:
f = open('ADA_USDT-3d.json')
data = json.load(f)
ada = pd.DataFrame(data)

In [185]:
f = open('BTC_USDT-3d.json')
data = json.load(f)
btc = pd.DataFrame(data)

In [186]:
f = open('DOGE_USDT-3d.json')
data = json.load(f)
doge = pd.DataFrame(data)

In [187]:
f = open('DOT_USDT-3d.json')
data = json.load(f)
dot = pd.DataFrame(data)

In [188]:
f = open('ETH_USDT-3d.json')
data = json.load(f)
eth = pd.DataFrame(data)

In [189]:
f = open('XRP_USDT-3d.json')
data = json.load(f)
xrp = pd.DataFrame(data)

In [190]:
ada.shape, btc.shape, doge.shape, dot.shape, eth.shape, xrp.shape

((56, 6), (56, 6), (56, 6), (56, 6), (56, 6), (56, 6))

In [191]:
price_data = [ada[4], btc[4], doge[4], eth[4], xrp[4]] 

In [192]:
def simplex_proj(y):
    """ Projection of y onto simplex. """
    m = len(y)
    bget = False

    s = sorted(y, reverse=True)
    tmpsum = 0.

    for ii in range(m-1):
        tmpsum = tmpsum + s[ii]
        tmax = (tmpsum - 1) / (ii + 1);
        if tmax >= s[ii+1]:
            bget = True
            break

    if not bget:
        tmax = (tmpsum + s[m-1] -1)/m
        
    y = np.asarray(y)

    return np.maximum(y-tmax,0.)


In [193]:
class OLMAR_VER2:
    def __init__(self, epsilon, window_size, assets):
        self.epsilon = epsilon
        self.window_size = window_size
        self.cumulative_wealth = 1.0 
        self.cash = 1000
        self.assets = assets

    def olmar(self, current_portfolio, predicted_returns, window_size):
        avg_x = np.sum([x*y for x,y in zip([1 for i in predicted_returns], predicted_returns)])
        avg_x *= 1.0 / float(len(predicted_returns))
        # numerator
        temp_right = self.epsilon - sum([x*y for x,y in zip(current_portfolio, predicted_returns)])
        # denominator
        temp_bottom1 = np.array(predicted_returns)
        temp_bottom2 = np.array([avg_x for i in predicted_returns])
        dist = np.sum(np.power((temp_bottom1 - temp_bottom2), 2))
        lambda_1 = max(0.0, temp_right/dist)
        # update portfolio
        np_current_portfolio = np.array(current_portfolio)
        new_portfolio = np_current_portfolio + lambda_1 * (temp_bottom1 - temp_bottom2)
        new_portfolio = simplex_proj(new_portfolio.tolist())
        return new_portfolio

    def predict(self, current_portfolio, current_return, previous_prices):
        # calculate cumulative wealth
        self.cumulative_wealth *= np.sum([x*y for x,y in zip(current_portfolio, current_return)])
        m = len(previous_prices[0]) # number of assets
        predicted_returns = []
        for i in range(m): # for each asset, calculate predicted return
            temp = 0
            for j in range(self.window_size):
                temp += previous_prices[j][i]
            predicted_return = (1/previous_prices[0][i]) * (1/self.window_size) * temp
            predicted_returns.append(predicted_return)
        
        new_portfolio = self.olmar(current_portfolio, predicted_returns, self.window_size) 
        actual_profits = self.calculate_actual_profits(current_portfolio, new_portfolio)

        return new_portfolio 

    def get_cumulative_wealth(self):
        return self.cumulative_wealth 
    
    def get_actual_profits(self): 
        return self.cash    
        
    def calculate_actual_profits(self, current_portfolio, new_portfolio, commission_rate=0.05):
        m = len(current_portfolio) 
        profits = [] 
        for i in range(m): 
            profits.append(previous_prices[0][i]/previous_prices[1][i])  
        
        actual_money = [current_portfolio[i] * self.cash * (1-0.05/100) for i in range(m)] 
        results = ([x*y for x,y in zip(actual_money, profits)])   
        after_commissions = 0 # money after closing all positions 
        for val in results: 
            after_commissions += val 
        
        after_commissions *= (1-0.05/100)  

        self.cash = after_commissions      

In [194]:
epsilon = 10 
m = 5 # number of assets 
w = 5
# create OLMAR instance 
olmar = OLMAR_VER2(epsilon, w, m)

iterations = 1 
current_portfolio = [1/m for i in range(m)] 


for i in range(w-1,len(price_data[0])):
    current_returns = [] 
    for j in range(m): 
        ret = price_data[j][i]/price_data[j][i-1] 
        current_returns.append(ret) 
    current_returns = np.asarray(current_returns) 
    
    previous_prices = [] 
    sub = 0 
    for j in range(w): 
        previous_price = [] 
        for k in range(m):
            previous_price.append(price_data[k][i-j])
        previous_prices.append(previous_price)  
        
    previous_prices = np.asarray(previous_prices)  
    
    new_portfolio = olmar.predict(current_portfolio, current_returns, previous_prices)  
    print("Iteration = {}".format(iterations))
    print("portfolio at time t = {}".format(current_portfolio))
    print("predicted portfolio at time t+1 = {}".format(new_portfolio))  
    
    print("actual profits = {}".format(olmar.get_actual_profits()))
    print()
    iterations += 1 
    
    current_portfolio = new_portfolio 
    


Iteration = 1
portfolio at time t = [0.2, 0.2, 0.2, 0.2, 0.2]
predicted portfolio at time t+1 = [0. 0. 1. 0. 0.]
actual profits = 1064.968027437323

Iteration = 2
portfolio at time t = [0. 0. 1. 0. 0.]
predicted portfolio at time t+1 = [0. 0. 1. 0. 0.]
actual profits = 1044.0756485964391

Iteration = 3
portfolio at time t = [0. 0. 1. 0. 0.]
predicted portfolio at time t+1 = [0. 1. 0. 0. 0.]
actual profits = 927.1053152779613

Iteration = 4
portfolio at time t = [0. 1. 0. 0. 0.]
predicted portfolio at time t+1 = [0. 1. 0. 0. 0.]
actual profits = 968.5023665267005

Iteration = 5
portfolio at time t = [0. 1. 0. 0. 0.]
predicted portfolio at time t+1 = [0. 0. 1. 0. 0.]
actual profits = 910.7370389117604

Iteration = 6
portfolio at time t = [0. 0. 1. 0. 0.]
predicted portfolio at time t+1 = [0. 1. 0. 0. 0.]
actual profits = 3468.741261303206

Iteration = 7
portfolio at time t = [0. 1. 0. 0. 0.]
predicted portfolio at time t+1 = [0. 1. 0. 0. 0.]
actual profits = 3586.9786082493206

Iteration

# simply holding

In [213]:
portfolio = [0.2,0.2,0.2,0.2,0.2] 
returns = [price_data[i][55] / price_data[i][0] * (1-0.05/100) for i in range(m)] 
result = np.sum([x*y for x,y in zip(portfolio, returns)]) 
initial_cash = 1000 

initial_cash*result

9057.153250547253